In [1]:
#Parametros
ancho = 10
alto = 10
nAgentes = 17
porcSucias = 5
tiempo = 100

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
#Numero celdas sucias
totalCeldas = ancho*alto
nSucias = ((totalCeldas) * porcSucias)/100
nLimpias = int(totalCeldas - nSucias)

In [4]:
#variables globales
#R1
stepCount = 0
#R2 
borradas = 0
#R3
movimientosCount = 0

from mesa import Agent
#clase limpiadores
class RoombaAgent(Agent):
    
    """ Modelo para un agente que se mueve aleatoriamente """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.direction = 2
       
        
    #Determina si se puede mover en la direccion elegida
    def move(self):
        #se guardara en la variable una lista con las posibles coordenadas (steps) por moverse
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True, # Boolean for whether to use Moore neighborhood (including diagonals) or Von Neumann (only up/down/left/right).
            include_center=True) 
        print(possible_steps)
        

        #existen len(possible_steps)-1 posibles movimientos, asi que toma una direccion
        #o sea una posicion del arraypossible_steps
        self.direction = self.random.randint(0,len(possible_steps)-1)
        print(f"direction antes de mover {self.direction}")
        
        #si existe la direccion haica cual se movera, se mueve
        if possible_steps[self.direction]:
            self.model.grid.move_agent(self, possible_steps[self.direction])
            global movimientosCount
            movimientosCount += 1
            print(f"Se mueve de {self.pos} a {possible_steps[self.direction]}; direction {self.direction}")
            
        else:
            print(f"No se puede mover de {self.pos} en esa direccion. {self.direction}")
        
        
    
    #se movera aleatoriamente porque en move se estaba usando random
    def step(self):
        self.move()
        
        
#las celdas sucias
class ObstacleAgent(Agent):
    """ Modelo para un Obstaculo """
    def __init__(self, unique_id,  model):
        super().__init__(unique_id, model)
        
        
    def step(self):
        #si esta en una posicion es True
        if self.pos:
            #si en la posicion en la que esta existe una clase limpiador, el agente se borra a si mismo
            for a in self.model.grid.get_cell_list_contents(self.pos):
                if(isinstance(a, RoombaAgent)):
                    self.model.grid.remove_agent(self)
                    self.model.schedule.remove(self)
                    global borradas 
                    borradas += 1
                    print("borrado: ",borradas)

    


In [5]:
#count  = 0
#modelo activa a los agentes
from mesa.time import RandomActivation
from mesa.space import Grid
from mesa import Model
from mesa.datacollection import DataCollector

class RandomModel(Model):
    
    """ Modelo para los autos """
    def __init__(self, nAgentes,ancho,alto):
        self.num_agents = nAgentes
        self.grid = Grid(ancho,alto,torus = False) 
        self.schedule = RandomActivation(self)
        self.running = True 
        
        global borradas
        global movimientosCount
        
        self.datacollector = DataCollector(
            {
                "Porcentaje de Limpias vs Steps": lambda m: ((nLimpias + borradas)*100)/totalCeldas,
    
                "Movimientos vs Steps": lambda m: movimientosCount
            }
        )


        #se pondra n numero de celdas sucias
        for i in range(int(nSucias)):
            # -----------Add the agent to a random empty grid cell--------- 
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            #mientras no estan vacias las coordenadas elegidas, buscara otras coordenadas para colocar agente
            while (not self.grid.is_cell_empty((x,y))):
                x = self.random.randrange(self.grid.width)
                y = self.random.randrange(self.grid.height) 
            a = ObstacleAgent(i+1000, self)
            self.schedule.add(a)
            self.grid.place_agent(a, (x, y))
            
        #iniciar agentes en (1,1)
        for i in range(self.num_agents):
            a = RoombaAgent(i+3000, self) #La numeracion de los agentes empieza en el 1000
            #se agrega agente al chedule
            self.schedule.add(a)
            #iniciar en (1,1)
            self.grid.place_agent(a, (1, 1))
        print("------ --------------------YA EMPEZO----------------------")
        
        self.datacollector.collect(self)
        
    def step(self):
        
        self.schedule.step()
        global stepCount
        stepCount +=1
        
        if(stepCount == tiempo-1):
            self.running = False
        #collect data
        
        global borradas
        global nSucias
        print("suciassss ",nSucias)
        print("borradasssss",borradas)
        
        if self.count_type(self) == 0:
            self.running = False
            
        self.datacollector.collect(self)

    
    @staticmethod
    def count_type(model):
        count = 0
        for agent in model.schedule.agents:
            if isinstance(agent, ObstacleAgent):
                count += 1
                print("count:::::::::::::::", count)
                
        return count
        

In [ ]:
#server
from mesa.visualization.modules import CanvasGrid,  ChartModule, PieChartModule
from mesa.visualization.ModularVisualization import ModularServer

#crear charts para los data collectors del modelo
tree_chart = ChartModule(
    [{"Label": "Porcentaje de Limpias vs Steps","Color": "red"}]
)
"Borrada vs Steps"
tree_chart2 = ChartModule(
    [{"Label": "Movimientos vs Steps" ,"Color": "blue"}]
)


def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 1,
                 "Color": "blue",
                 #radius creo
                 "r": 0.5}
    
    if (isinstance(agent,RoombaAgent)):
        portrayal["Color"] = "blue"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.5
    
    #si esta sucio
    if (isinstance(agent,ObstacleAgent)):
        portrayal["Color"] = "brown"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.5
    
    return portrayal
    

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(RandomModel,
                       [grid, tree_chart, tree_chart2],
                       "Roomba",
                       {"nAgentes":nAgentes, "ancho":ancho, "alto":alto})
server.port = 1536 # The default
server.launch()

------ --------------------YA EMPEZO----------------------
Interface starting at http://127.0.0.1:1536
Socket opened!
{"type":"reset"}
------ --------------------YA EMPEZO----------------------
{"type":"get_step","step":1}
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 3
Se mueve de (1, 0) a (1, 0); direction 3
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 6
Se mueve de (2, 0) a (2, 0); direction 6
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 8
Se mueve de (2, 2) a (2, 2); direction 8
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 2
Se mueve de (0, 2) a (0, 2); direction 2
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 1
Se mueve de (0, 1) a (0, 1); direction 1
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

{"type":"get_step","step":5}
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 2
Se mueve de (0, 2) a (0, 2); direction 2
[(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
direction antes de mover 8
Se mueve de (2, 3) a (2, 3); direction 8
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
direction antes de mover 0
Se mueve de (0, 0) a (0, 0); direction 0
[(3, 0), (3, 1), (4, 0), (4, 1), (5, 0), (5, 1)]
direction antes de mover 2
Se mueve de (4, 0) a (4, 0); direction 2
[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]
direction antes de mover 5
Se mueve de (2, 1) a (2, 1); direction 5
[(0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4)]
direction antes de mover 7
Se mueve de (2, 3) a (2, 3); direction 7
[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]
direction antes de mover 1
Se mueve de (0, 1) a (0, 1); direction 1
[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]
direction antes de mover 3
Se m

{"type":"get_step","step":9}
[(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5)]
direction antes de mover 5
Se mueve de (1, 5) a (1, 5); direction 5
[(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3)]
direction antes de mover 3
Se mueve de (1, 1) a (1, 1); direction 3
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 2
Se mueve de (0, 2) a (0, 2); direction 2
[(0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4)]
direction antes de mover 5
Se mueve de (1, 4) a (1, 4); direction 5
[(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
direction antes de mover 5
Se mueve de (1, 3) a (1, 3); direction 5
[(0, 0), (0, 1), (1, 0), (1, 1)]
direction antes de mover 0
Se mueve de (0, 0) a (0, 0); direction 0
[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]
direction antes de mover 2
Se mueve de (1, 3) a (1, 3); direction 2
[(2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3), (4, 1), (4, 2), (4, 3)]
direction antes de move

{"type":"get_step","step":13}
[(2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2)]
direction antes de mover 2
Se mueve de (2, 2) a (2, 2); direction 2
[(2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2)]
direction antes de mover 3
Se mueve de (3, 0) a (3, 0); direction 3
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 5
Se mueve de (1, 2) a (1, 2); direction 5
[(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)]
direction antes de mover 6
Se mueve de (2, 3) a (2, 3); direction 6
[(0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4)]
direction antes de mover 0
Se mueve de (0, 2) a (0, 2); direction 0
[(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5)]
direction antes de mover 2
Se mueve de (0, 5) a (0, 5); direction 2
[(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)]
direction antes de mover 4
Se mueve de (4, 4) a (4, 4); direction 4
[(0, 5), (0, 6), (0, 7), (1, 5)

{"type":"get_step","step":17}
[(0, 5), (0, 6), (0, 7), (1, 5), (1, 6), (1, 7)]
direction antes de mover 2
Se mueve de (0, 7) a (0, 7); direction 2
[(4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (6, 0), (6, 1), (6, 2)]
direction antes de mover 7
Se mueve de (6, 1) a (6, 1); direction 7
[(2, 2), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4), (4, 2), (4, 3), (4, 4)]
direction antes de mover 7
Se mueve de (4, 3) a (4, 3); direction 7
[(0, 8), (0, 9), (1, 8), (1, 9), (2, 8), (2, 9)]
direction antes de mover 0
Se mueve de (0, 8) a (0, 8); direction 0
[(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
direction antes de mover 3
Se mueve de (1, 1) a (1, 1); direction 3
[(1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2)]
direction antes de mover 8
Se mueve de (3, 2) a (3, 2); direction 8
[(5, 0), (5, 1), (5, 2), (6, 0), (6, 1), (6, 2), (7, 0), (7, 1), (7, 2)]
direction antes de mover 3
Se mueve de (6, 0) a (6, 0); direction 3
[(2, 3), (2, 4), (2, 5), (3, 3)

{"type":"get_step","step":21}
[(1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2), (3, 0), (3, 1), (3, 2)]
direction antes de mover 3
Se mueve de (2, 0) a (2, 0); direction 3
[(0, 0), (0, 1), (1, 0), (1, 1)]
direction antes de mover 3
Se mueve de (1, 1) a (1, 1); direction 3
[(0, 6), (0, 7), (0, 8), (1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8)]
direction antes de mover 4
Se mueve de (1, 7) a (1, 7); direction 4
[(1, 7), (1, 8), (1, 9), (2, 7), (2, 8), (2, 9), (3, 7), (3, 8), (3, 9)]
direction antes de mover 8
Se mueve de (3, 9) a (3, 9); direction 8
[(3, 0), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2)]
direction antes de mover 2
Se mueve de (3, 2) a (3, 2); direction 2
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 6
Se mueve de (2, 0) a (2, 0); direction 6
[(0, 3), (0, 4), (0, 5), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5)]
direction antes de mover 1
Se mueve de (0, 4) a (0, 4); direction 1
[(4, 0), (4, 1), (4, 2)

{"type":"get_step","step":25}
[(3, 7), (3, 8), (3, 9), (4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9)]
direction antes de mover 0
Se mueve de (3, 7) a (3, 7); direction 0
[(0, 6), (0, 7), (0, 8), (1, 6), (1, 7), (1, 8)]
direction antes de mover 2
Se mueve de (0, 8) a (0, 8); direction 2
[(4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9)]
direction antes de mover 3
Se mueve de (5, 7) a (5, 7); direction 3
[(0, 5), (0, 6), (0, 7), (1, 5), (1, 6), (1, 7), (2, 5), (2, 6), (2, 7)]
direction antes de mover 2
Se mueve de (0, 7) a (0, 7); direction 2
[(0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)]
direction antes de mover 5
Se mueve de (2, 1) a (2, 1); direction 5
[(3, 1), (3, 2), (3, 3), (4, 1), (4, 2), (4, 3), (5, 1), (5, 2), (5, 3)]
direction antes de mover 4
Se mueve de (4, 2) a (4, 2); direction 4
[(1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4)]
direction antes de mover 2
Se mueve de (1, 4) a (1, 4); direction 2
[(3, 4), (3, 5), (3, 6), (4, 4)

{"type":"get_step","step":29}
[(3, 8), (3, 9), (4, 8), (4, 9), (5, 8), (5, 9)]
direction antes de mover 0
Se mueve de (3, 8) a (3, 8); direction 0
[(6, 0), (6, 1), (6, 2), (7, 0), (7, 1), (7, 2), (8, 0), (8, 1), (8, 2)]
direction antes de mover 3
Se mueve de (7, 0) a (7, 0); direction 3
[(1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8), (3, 6), (3, 7), (3, 8)]
direction antes de mover 0
Se mueve de (1, 6) a (1, 6); direction 0
[(4, 2), (4, 3), (4, 4), (5, 2), (5, 3), (5, 4), (6, 2), (6, 3), (6, 4)]
direction antes de mover 0
Se mueve de (4, 2) a (4, 2); direction 0
[(1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4)]
direction antes de mover 0
Se mueve de (1, 2) a (1, 2); direction 0
[(0, 1), (0, 2), (0, 3), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
direction antes de mover 0
Se mueve de (0, 1) a (0, 1); direction 0
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 8
Se mueve de (2, 2) a (2, 2); direction 8
[(0, 6)

{"type":"get_step","step":33}
[(0, 6), (0, 7), (0, 8), (1, 6), (1, 7), (1, 8), (2, 6), (2, 7), (2, 8)]
direction antes de mover 4
Se mueve de (1, 7) a (1, 7); direction 4
[(1, 8), (1, 9), (2, 8), (2, 9), (3, 8), (3, 9)]
direction antes de mover 2
Se mueve de (2, 8) a (2, 8); direction 2
[(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)]
direction antes de mover 4
Se mueve de (4, 4) a (4, 4); direction 4
[(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)]
direction antes de mover 6
Se mueve de (3, 4) a (3, 4); direction 6
[(2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3), (4, 1), (4, 2), (4, 3)]
direction antes de mover 6
Se mueve de (4, 1) a (4, 1); direction 6
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
direction antes de mover 0
Se mueve de (0, 0) a (0, 0); direction 0
[(5, 0), (5, 1), (6, 0), (6, 1), (7, 0), (7, 1)]
direction antes de mover 3
Se mueve de (6, 1) a (6, 1); direction 3
[(0, 3), (0, 4), (0, 5), (1, 3)

{"type":"get_step","step":37}
